<a href="https://colab.research.google.com/github/SLCFLAB/Data-Science-Python/blob/main/Day%2012/12_3_rnn_pytorch_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 32
num_epochs = 2
learning_rate = 0.01

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/2], Step [100/1875], Loss: 1.1766
Epoch [1/2], Step [200/1875], Loss: 0.7775
Epoch [1/2], Step [300/1875], Loss: 0.2570
Epoch [1/2], Step [400/1875], Loss: 0.3504
Epoch [1/2], Step [500/1875], Loss: 0.3229
Epoch [1/2], Step [600/1875], Loss: 0.6161
Epoch [1/2], Step [700/1875], Loss: 0.1609
Epoch [1/2], Step [800/1875], Loss: 0.2335
Epoch [1/2], Step [900/1875], Loss: 0.0813
Epoch [1/2], Step [1000/1875], Loss: 0.1825
Epoch [1/2], Step [1100/1875], Loss: 0.0448
Epoch [1/2], Step [1200/1875], Loss: 0.2550
Epoch [1/2], Step [1300/1875], Loss: 0.3437
Epoch [1/2], Step [1400/1875], Loss: 0.1161
Epoch [1/2], Step [1500/1875], Loss: 0.1274
Epoch [1/2], Step [1600/1875], Loss: 0.1710
Epoch [1/2], Step [1700/1875], Loss: 0.1964
Epoch [1/2], Step [1800/1875], Loss: 0.0520
Epoch [2/2], Step [100/1875], Loss: 0.0136
Epoch [2/2], Step [200/1875], Loss: 0.2031
Epoch [2/2], Step [300/1875], Loss: 0.0138
Epoch [2/2], Step [400/1875], Loss: 0.0835
Epoch [2/2], Step [500/1875], Loss: 0.3713
Ep